# Introduction to TD and SARSA-Learning 

## CSCI E-82A

## Stephen Elston


In the previous lesson we explored Monte Carlo **reinforcement learning**. MC RL required that the returns for an entire episode be computed before any values are available for use. The disadvantage of this approach is the the full set of returns are required for state value or action value estimates. But, how can we get state values or action-values in fewer time steps? It turns out there are algorithms which compute estimates in as few as one step. In this lesson will focus on a state-value estimation algorithm known as **time difference learning** or **TD-learning** and control algorithm known as **SARSA**. 

Recall that reinforcement learning has several distinctive characteristics, which differentiate this method from other machine learning and dynamic programming:
- **No Markov model** needs to be specified for reinforcement learning, in contrast to dynamic programming.
- Like dynamic programming, reinforcement learning **optimizes a reward function**. This is in contrast to supervised and unsupervised learning which use an error or objective function.  
- Reinforcement learning algorithms learn by **experience**. Over time, the algorithm learns a model of the environment and these results are used to optimize the expected reward. Learning from experience is in contrast to supervised learning which uses known marked cases. 
- Reinforcement learning agents take **actions** and only receive **state** and **rewards** from the environment. These are the only interaction between the RL agent and the environment.    

The interaction between a reinforcement learning agent and the environment are illustrated in the figure below. Notice that the only feedback the agent receives from the environment is reward and state.   

<img src="img/RL_AgentModel.JPG" alt="Drawing" style="width:500px; height:300px"/>
<center> **Reinforcement Learning Agent and Environment** </center>  

The ability to learn from experience is an attractive concept. This method of learning seems to mimic human learning. However, reinforcement learning has proven difficult to use in real-world applications. For a review of successes and problems arising when applying RL to robotics see [Kobler et. al.](https://www.ias.informatik.tu-darmstadt.de/uploads/Publications/Kober_IJRR_2013.pdf). At the present time, RL has mostly succeeded in cases where simulations can be used to gain experience. 

**Suggested readings** for TD and Q reinforcement learning, Chapters 6 and 7 of Sutton and Barto, second edition, provides a good introductions, including many alternative algorithms and details not discussed here.   

## TD Prediction Model

In a previous lesson we examined a general update model:

$$NewValue = OldValue + LearningRate * ErrorTerm$$

Following this general formulation the update for TD state value can be written as:

$$V(S_t) = V(S_t) + \alpha \big[ G_T - V(S_t) \big]$$

Here,   
$\alpha = $ the learning rate,    
$\big[ G_T - V(S_t) \big] = $ the error term is the difference between the return $G_t$ and the value, $V(S_t)$. The return is identical to the value at convergence.   

   
Using the same general formulation we can create and single time step update model know as the **one step time difference** or **TD(0)** algorithm.

$$V(S_t) = V(S_t) + \alpha \big[ R_{t+1} + V(S_{t+1}) - V(S_t) \big]$$

Where,  
$\delta_t =  R_{t+1} + V(S_{t+1}) - V(S_t) = $ the **TD error**,  
$R_{t+1} = $ the return for the next time step,   
$V(S_t) = $ is the value at time step t.   

Like dynamic programming algorithms, the TD algorithm **bootstraps**. The return and estimated value at the next time step, $V(S_{t+1})$, are from previous samples. However, unlike MC RL which does not bootstrap, the TD(0) algorithm produces an estimate of $V(S_t)$ in only one time step, rather than waiting to reach the terminal state at the end of the episode. This fact can be seen by examining the backup diagram shown below:

<img src="img/TD0.JPG" alt="Drawing" style="width:60px; height:150px"/>
<center> **Backup Diagram of TD(0)** </center>

## Example of Time Difference RL

With this short introduction TD RL in mind, let's try an example. We will sample the value function using a basic TD(0) algorithm here. 

As discussed in other labs, **Navigation** to a goal is a significant problem in robotics. Real-world navigation is rather complex. Therefore, in this example we will use a simple analog called a **grid world**. The grid world for this problem is shown below. 

<img src="img/GridWorld.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **A 4x4 Grid World with Terminal State** </center>

The grid world consists of a 4x4 set of positions the robot can occupy. Each position is considered a state. The goal is to navigate to state 0, the goal, in the minimum steps. We will explore methods to find policies which reach this goal and achieve maximum reward. 

Grid position 0 is the goal and a **terminal state**. There are no possible state transitions out of this position. The presence of a terminal state makes this an **episodic Markov random process**. For each episode sampled the robot can start in any other random position, $\{ 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15 \}$. This random selection process makes this a **random start** TD algorithm. The episode terminates when the robot enters the terminal position (state 0).  

In reality, an RL agent may need to explore to find the possible actions when it is in some particular state. To simplify our example, we encode, or represent, these possibilities in a dictionary as shown in the code block below. We use a dictionary of dictionaries to perform the lookup. The keys of the outer dictionary are the identifiers (numbers) of the states. The keys of the inner dictionary are the possible actions and the values are the **successor state**, $s'$, for that transition.  

In each state, there are four possible actions the robot can take:
- up, u
- down, d,
- left, l
- right, r

The TD RL agent has no model for the environment. Therefore, beyond these allowed actions, all other information is encapsulated in the environment and is unobservable by the agent. This is the key difference between reinforcement learning and dynamic programming. 

In [2]:
## import numpy for latter
import numpy as np
import numpy.random as nr
import pandas as pd

## Define the transition dictonary of dictionaries:
neighbors = {0:{'u':0, 'd':0, 'l':0, 'r':0},
          1:{'u':1, 'd':5, 'l':0, 'r':2},
          2:{'u':2, 'd':6, 'l':1, 'r':3},
          3:{'u':3, 'd':7, 'l':2, 'r':3},
          4:{'u':0, 'd':8, 'l':4, 'r':5},
          5:{'u':1, 'd':9, 'l':4, 'r':6},
          6:{'u':2, 'd':10, 'l':5, 'r':7},
          7:{'u':3, 'd':11, 'l':6, 'r':7},
          8:{'u':4, 'd':12, 'l':8, 'r':9},
          9:{'u':5, 'd':13, 'l':8, 'r':10},
          10:{'u':6, 'd':14, 'l':9, 'r':11},
          11:{'u':7, 'd':15, 'l':10, 'r':11},
          12:{'u':8, 'd':12, 'l':12, 'r':13},
          13:{'u':9, 'd':13, 'l':12, 'r':14},
          14:{'u':10, 'd':14, 'l':13, 'r':15},
          15:{'u':11, 'd':15, 'l':14, 'r':15}}

To simulate the environment, we need a reward structure. In this case, the robot receives the following rewards:   

- 10 for entering position 0. 
- -1 for attempting to leave the grid. In other words, we penalize the robot for hitting the edges of the grid.  
- -0.1 for all other state transitions, which is the cost for the robot to move from one state to another. If we did not have this penalty, the robot could follow any random plan to the goal which did not hit the edges. 

This **reward structure is unknown to the TD RL agent**. The agent must **learn** the rewards by sampling the environment. Here the rewards are in the form of action values.    

We encode these rewards in the same type of dictionary structure used for the foregoing structures. 

In [3]:
rewards = {0:{'u':10.0, 'd':10.0, 'l':10.0, 'r':10.0},
          1:{'u':-1, 'd':-0.1, 'l':10.0, 'r':-0.1},
          2:{'u':-1.0, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          3:{'u':-1.0, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          4:{'u':10.0, 'd':-0.1, 'l':-1.0, 'r':-0.1},
          5:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          6:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          7:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          8:{'u':-0.1, 'd':-0.1, 'l':-1.0, 'r':-0.1},
          9:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          10:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          11:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          12:{'u':-0.1, 'd':-1.0, 'l':-1.0, 'r':-0.1},
          13:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-0.1},
          14:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-0.1},
          15:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-1.0}}

As was done previously, we will use an environment simulator. The function is called with a state and action. It returns the next state, 's_prime' and 'reward'. 

To simplify the rest of the code in this notebook we are treating the dictionaries as global. In general, this would be considered poor programming practice. 

Execute the code in the cell below and observe the results from the test cases. 

In [4]:
def simulate_environment(s, action, neighbors = neighbors, rewards = rewards, terminal = 0):
    """
    Function simulates the environment
    returns s_prime and reward given s and action
    """
    s_prime = neighbors[s][action]
    reward = rewards[s][action]
    return (s_prime, reward, is_terminal(s_prime, terminal))

def is_terminal(state, terminal = 0):
    return state == terminal

## Test the function
for a in ['u', 'd', 'r', 'l']:
    print(simulate_environment(1, a))

(1, -1, False)
(5, -0.1, False)
(2, -0.1, False)
(0, 10.0, True)


### TD(0) Policy Evaluation

We have everything in place to perform TD(0) policy evaluation for the grid world. The code in the cell below implements the TD(0) algorithm and applies it to the policy in the grid world. Notice that the algorithm makes calls to the aforementioned `state_values` function to find information on successor state and rewards for a transition given a current state and action. Execute this code and examine the results. 

In [5]:
initial_policy = {0:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        1:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25}, 
                        2:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        3:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        4:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        5:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        6:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        7:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        8:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        9:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        10:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        11:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        12:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        13:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        14:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        15:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25}}

In [6]:
def start_episode(n_states):
    '''Function to find a random starting value for the episode
    that is not the terminal state'''
    state = nr.choice(range(n_states))
    while(is_terminal(state)):
         state = nr.choice(range(n_states))
    return state

## test the function to make sure never starting in terminal state
[start_episode(15) for _ in range(10)]

[9, 11, 2, 3, 10, 13, 14, 10, 8, 4]

In [7]:
def take_action(state, policy, actions = {1:'u', 2:'d', 3:'l', 4:'r'}):
    '''Function takes action given state using the transition probabilities 
    of the policy'''
    ## Find the action given the transistion probabilities defined by the policy.
    action = actions[nr.choice(range(len(actions)), p = list(policy[state].values())) + 1]
    s_prime, reward, terminal = simulate_environment(state, action)
    return (action, s_prime, reward, terminal)

## Test function for several states
for s in range(16):
    print(take_action(s, initial_policy))

('d', 0, 10.0, True)
('l', 0, 10.0, True)
('l', 1, -0.1, False)
('r', 3, -1.0, False)
('u', 0, 10.0, True)
('l', 4, -0.1, False)
('d', 10, -0.1, False)
('l', 6, -0.1, False)
('u', 4, -0.1, False)
('d', 13, -0.1, False)
('r', 11, -0.1, False)
('r', 11, -1.0, False)
('u', 8, -0.1, False)
('u', 9, -0.1, False)
('u', 10, -0.1, False)
('d', 15, -1.0, False)


In [8]:
def td_0_state_values(policy, n_samps, alpha = 0.2, gamma = 1.0):
    """
    Function for TD(0) policy evalutation
    """
    
    ## Find the starting state
    n_states = len(policy)
    current_state = start_episode(n_states)
    terminal = False
    
    ## Array for state values
    v = np.zeros((n_states,1))
    
    for _ in range(n_samps):
        ## Find the next action and reward
        action, s_prime, reward, terminal = take_action(current_state, policy)
        ## Compute the TD error
        delta = reward + gamma*v[s_prime] - v[current_state]
        ## Update the state value
        v[current_state] = v[current_state] + alpha*delta
        current_state = s_prime
        if(terminal): ## start new episode when terminal
            current_state = start_episode(n_states)
    return(v)

td_0_state_values(initial_policy, 2000).reshape((4,4))        

array([[ 0.        ,  5.04765399, -0.20524413, -2.65144204],
       [ 3.520735  ,  1.08490573, -1.23443367, -3.04394341],
       [ 1.1215535 , -0.78622549, -1.46191424, -3.11343686],
       [-1.84278949, -2.36107651, -2.79733567, -4.65273613]])

## One Step SARSA Algorithm

Now that we have examined the one step TD(0) algorithm for policy (value) evaluation, we need to define a one step algorithm for **control** or **policy improvement**.     

The **state action reward state action** or **SARSA** algorithm is an **on policy** method which uses time differencing to evaluate action values. As you might imagine from the name, this algorithm starts with an **on policy** action from the current state which results in a state transition and a reward. The backup diagram for one step SARSA (SARSA(0)) is shown in the figure below. 

<img src="img/SARSA.JPG" alt="Drawing" style="width:75px; height:150px"/>
<center> **Backup Diagram for SARSA(0)** </center>

Compare the backup diagram for SARSA(0) to the one for TD(0) notice that the order of state and action are reversed between the two algorithms. This realization is a good way to understand the difference. 

The update equation for SARSA(0) is as follows:

$$Q(S_t,A_t) = Q(S_t,A_t) + \alpha \big[ R_{t+1} + \gamma Q(S_{t+1},A_{t+1}) - Q(S_t,A_t) \big]$$  

Where,   
$\delta_t = R_{t+1} + \gamma Q(S_{t+1},A_{t+1}) - Q(S_t,A_t) = $ the error term,   
$Q(S_t,A_t) = $ is the action value in state S given action A,  
$R_{t+1} = $ is the reward for the next time step,   
$\alpha = $ the learning rate,   
$\gamma = $ discount factor.  

### SARSA(0) Example

The code in the cell below implements the SARSA(0) algorithm to compute the action values for the Grid world given an policy. As with the policy evaluation code, this code makes calls to the environment t find successor states and rewards given actions. The function `select_a_prime` queries the environment to receive the successor state and reward given the current state and action. The successor action is computed following the policy. Additional details on this algorithm can be seen by reading the code comments.  

Execute this code to compute and print the action values for the random walk policy.     

In [9]:
def print_Q(Q):
    Q = pd.DataFrame(Q, columns = ['up', 'down', 'left', 'right'])
    print(Q)

def new_episode(n_states, policy):
    '''This function provides a start for a TD
    episode making sure the first transition is not 
    the termnal state'''
    current_state = start_episode(n_states)
    ## Find fist action and reward
    action, s_prime, reward, terminal = take_action(current_state, policy)
    return(current_state, action, s_prime, reward, terminal)    


def SARSA_0(policy, n_samps, alpha = 0.1, gamma = 0.9, action_index = {'u':0, 'd':1, 'l':2, 'r':3}):
    """
    Function for TD(0) policy evalutation
    """
    
    ## Find the starting state
    n_states = len(policy)
    current_state, action, s_prime, reward, terminal = new_episode(n_states, policy)
    action_idx = action_index[action]
    
    ## Array for state values
    q = np.zeros((n_states, len(policy[0])))
    
    for _ in range(n_samps):
        ## Find the next action and reward
        action_prime, s_prime_prime, reward_prime, terminal_prime = take_action(s_prime, policy)
        action_idx_prime = action_index[action_prime]
        ## Compute the TD error
        delta = reward + gamma*q[s_prime, action_idx_prime] - q[current_state, action_idx]
        ## Update the action values
        q[current_state, action_idx] = q[current_state, action_idx] + alpha*delta
        ## Update the state, action and reward for the next time step
        current_state = s_prime
        s_prime = s_prime_prime
        action = action_prime
        reward = reward_prime
        terminal = terminal_prime
        action_idx = action_idx_prime

        ## Check if end of episode
        if(terminal): 
            ## start new episode
            current_state, action, s_prime, reward, terminal = new_episode(n_states, policy)        
    return(q)


Q = SARSA_0(initial_policy, 20000, alpha = 0.2, gamma = 0.99)
print_Q(Q)

           up       down       left      right
0    0.000000   0.000000   0.000000   0.000000
1   -7.088180  -8.555462  -4.299538  -8.828404
2  -10.451587  -8.852504  -8.284065 -10.814911
3  -11.292410 -10.051300  -9.591803 -11.168060
4   -2.744298  -9.126567  -8.183728  -8.077597
5   -7.528405  -9.114228  -7.709337  -9.069458
6   -9.363227  -9.728455  -8.185111  -9.868513
7  -10.797907 -10.231357  -9.023329 -10.997462
8   -6.927557 -10.226893 -10.415021  -9.455506
9   -8.066996  -9.656808  -8.848401  -9.784698
10  -9.195068 -10.522548  -8.743939 -10.146415
11 -10.147279 -10.668184  -9.622402 -11.020557
12  -9.864709 -10.951511 -10.649736  -9.654366
13  -8.587686 -10.486233 -10.189523 -10.262943
14  -9.683353 -11.090443 -10.214260 -11.797894
15 -10.047831 -12.625380 -10.590217 -11.741864


The arrays printed above display the action values for each state. The four arrays represent the four possible actions, up, down, left and right. 

### GPI with SARSA(0)

The code in the cell below performs **general policy improvement (GPI)** using SARSA(0) to evaluate action values. Several cycles of SARSA(0) and policy improvement are performed with this code. Notice that policy improvement is $\epsilon$-greedy. The probability of any state transition will never go to zero, allowing continued exploration. Additional details on this algorithm can be seen by reading the code comments.  

Execute this code and examine the results.  

In [10]:
def update_policy(policy, Q, epsilon, action_index = {'u':0, 'd':1, 'l':2, 'r':3}):
    '''Updates the policy based on estiamtes of Q using 
    an epslion greedy algorithm. The action with the highest
    action value is used.'''
    
    ## Find the keys for the actions in the policy
    keys = list(policy[0].keys())
    
    ## Iterate over the states and find the maximm action value.
    for state in range(len(policy)):
        ## First find the index of the max Q values  
        q = Q[state,:]
        max_action_index = np.where(q == max(q))[0]

        ## Find the probabilities for the transitions
        n_transitions = float(len(q))
        n_max_transitions = float(len(max_action_index))
        p_max_transitions = (1.0 - epsilon *(n_transitions - n_max_transitions))/(n_max_transitions)
  
        ## Now assign the probabilities to the policy as epsilon greedy.
        for key in keys:
            if(action_index[key] in max_action_index): policy[state][key] = p_max_transitions
            else: policy[state][key] = epsilon
    return(policy)                

update_policy(initial_policy, Q, 0.1)    

{0: {'d': 0.25, 'l': 0.25, 'r': 0.25, 'u': 0.25},
 1: {'d': 0.1, 'l': 0.7, 'r': 0.1, 'u': 0.1},
 2: {'d': 0.1, 'l': 0.7, 'r': 0.1, 'u': 0.1},
 3: {'d': 0.1, 'l': 0.7, 'r': 0.1, 'u': 0.1},
 4: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 5: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 6: {'d': 0.1, 'l': 0.7, 'r': 0.1, 'u': 0.1},
 7: {'d': 0.1, 'l': 0.7, 'r': 0.1, 'u': 0.1},
 8: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 9: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 10: {'d': 0.1, 'l': 0.7, 'r': 0.1, 'u': 0.1},
 11: {'d': 0.1, 'l': 0.7, 'r': 0.1, 'u': 0.1},
 12: {'d': 0.1, 'l': 0.1, 'r': 0.7, 'u': 0.1},
 13: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 14: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 15: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7}}

In [11]:
def SARSA_GPI(policy, n_samples, n_cycles, epsilon = 0.1, n_actions = 4):
    '''Function perfoms GPI using Monte Carlo value estimation.
    Updates to policy are epsilon greedy to prevent the algorithm
    from being trapped at some point.'''
    Q = np.zeros((len(policy), n_actions))
    ## Iterate over the required number of cycles
    for _ in range(n_cycles):
        Q = SARSA_0(policy, n_samples, alpha = 0.2, gamma = 0.99)
        policy = update_policy(policy, Q, epsilon = epsilon)
    return(policy)

improved_policy = SARSA_GPI(initial_policy, 100, 50, epsilon = 0.1)  
for state in range(16):
    print(improved_policy[state])

{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.1, 'd': 0.7, 'l': 0.1, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.1, 'd': 0.1, 'l': 0.4, 'r': 0.4}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.1, 'd': 0.7, 'l': 0.1, 'r': 0.1}
{'u': 0.1, 'd': 0.7, 'l': 0.1, 'r': 0.1}
{'u': 0.4, 'd': 0.4, 'l': 0.1, 'r': 0.1}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.1, 'd': 0.3, 'l': 0.3, 'r': 0.3}
{'u': 0.1, 'd': 0.4, 'l': 0.1, 'r': 0.4}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}


## N-Step Time Differencing

In the previous lesson we examined Monte Carlo RL which requires that each episode reach a terminal state before a value or action value estimate can be made. In the first part of this lesson we have focused on one-step time differencing, TD, algorithms which update estimates at each time step. Now, we will look at algorithms between these two extreme cases, n-step time differencing algorithms. 

We can generalize the concept of the one-step bootstrapping of the return starting with TD(0):

$$G_{t:t+1} = R_{t+1} + \gamma V_{t}(S_{t+1})$$

Here, $V_{t}(S_{t+1})$ is being bootstrapped. 

We can extend this formulation to two-step TD as follows:

$$G_{t:t+2} = R_{t+1} + \gamma R_{t+2} + \gamma^2 V_{t+1}(S_{t+2})$$

In the above it is $V_{t+1}(S_{t+2})$ being bootstrapped. 

For n-step TD the return is computed by the following bootstrap formulation. 

$$G_{t:t+n} = R_{t+1} + \gamma R_{t+2} + \ldots + \gamma^{n-1} R_{t+n} + \gamma^n V_{t+n-1}(S_{t+n})$$

Regardless of home many steps are used to compute the return, the value update is:

$$V_{t+n}(S_t) = V_{t+n-1}(S_t) + \alpha \big[ G_{t:t+n} - V_{t+n-1}(S_t) \big],\ 0 \leq t < T]$$

Where $\delta_t = G_{t:t+n} - V_{t+n-1}(S_t) = $ the n-step TD error.

The resulting backup diagrams for general TD(n) is shown in the diagram below. 

<img src="img/TDN.JPG" alt="Drawing" style="width:400px; height:350px"/>
<center> **Backup Diagram for n-step TD** </center>

From Left to right you can see how the backup diagram adds more states and actions. On the extreme right the backup diagram for the MC algorithm is shown. Notice that MC RL does not bootstrap, but samples until the terminal state is reached. 

N-step TD methods are generally considered to have two advantages over single-step methods:
1. N-step methods generally have better convergence properties. 
2. N-step methods break the link between time between actions and time when bootstraping occurs.   

### Example of TD(n) for Policy Evaluation

The code in the cell below implements TD(n) policy evaluation. As you can see, the bookkeeping is a bit involved. We must account for the cases where fewer than n steps have been executed at the start of an episode or when the terminal state has been reached. You can find further details of the algorithm by reading the code comments. 

Execute this code for n=4 and examine the results.

In [15]:
def TD_n(policy, episodes, n, alpha = 0.1, gamma = 0.9): 
    """
    Function to perform TD(N) policy evaluation.
    """
    ## Initialize the state list and action values
    states = list(policy.keys())
    n_states = len(states)
    v = np.zeros((n_states))
    
    for _ in range(episodes):
        ## Initialize variables
        T = float("inf")
        tau = 0
        t = 0
        terminal = False
        rewards = []
       
        ## Get the random initial state
        current_state = start_episode(n_states)   
        state = [current_state]
        ## Initial action
#        action, s_prime, reward, terminal = take_action(current_state, policy)
#        state.append(s_prime)
        
#        while(not (tau == T - 1)):
        while(not terminal):
#            print('tau = {}    T = {}      t = {}'.format(tau,T,t))
            if(t < T):
#                ## Append the reward to the list
#                rewards.append(reward)
#                ## Get the next action state and rewards
#                action_prime, s_prime_prime, reward_prime, terminal_prime = take_action(current_state, policy)
#                state.append(s_prime_prime)
                action, s_prime, reward, terminal = take_action(current_state, policy)
                state.append(s_prime)
                ## Append the reward to the list
                rewards.append(reward)
                
                if(terminal): ## update T if at terminal state
                    T = t + 1
 #               else: 
 #                   ## Get the next action state and rewards
 #                   action_prime, s_prime_prime, reward_prime, terminal_prime = take_action(current_state, policy)
 #                   state.append(s_prime_prime)  
 #                   ## Append the reward to the list
 #                   rewards.append(reward)
                    
            ## Update tau
            tau = t - n + 1
            
#            print('')
#            print('length of rewards = {}'.format(len(rewards)) )
#            print('length of state = {}'.format(len(state)) )
#            print('tau = {}    T = {}      t = {}'.format(tau,T,t))
#            print('s = {}  s_prime = {}   action = {}'.format(current_state,s_prime,action))
#            print(rewards)
            G = 0.0
            if(tau >= 0):
                for i in range(tau + 1, min(tau + n, T)):
                    exponent = i + tau - 1
                    G = G + pow(gamma, exponent) * rewards[i]
#                    print('i = {}  exponent = {}   G = {}   rewards[i] =  '.format(i, exponent, G, rewards[i]))
                    if((tau + n) < T): G += pow(gamma,n) * v[state[tau + n]] 
                    v[state[tau]] = v[state[tau]] + alpha * (G - v[state[tau]])    
                
            
            ## Update variables for the next step
 #           if(not terminal):
            t += 1
#            action = action_prime
#            s_prime = s_prime_prime
            current_state = s_prime
#            reward = reward_prime
#            terminal = terminal_prime
    return(v)              
            
TD_n(initial_policy, 1000, 2).reshape((4,4))            

array([[ 0.        ,  0.08774282,  0.17904768,  0.07213563],
       [ 0.02052932,  0.10033007,  0.05101877,  0.07569612],
       [-0.03450299, -0.0691556 , -0.05063009,  0.02052463],
       [-0.07830954, -0.07600302, -0.0046366 ,  0.00103352]])

## N-step SARSA for control

Just as we used SARSA(0) for the control or policy improvement, we can generalize this algorithm to the n-step case. This is the SARSA(n) algorithm.  

The n-step return is computed by the following bootstrap formulation. 

$$G_{t:t+n} = R_{t+1} + \gamma R_{t+2} + \ldots + \gamma^{n-1} R_{t+n} + \gamma^n Q_{t+n-1}(S_{t+n}, A_{t+n}),\ n \geq 1, 0 \leq t < T-n$$

Using this return, the action value update becomes:

$$Q_{t+n}(S_t, A_t) = Q_{t+n-1}(S_t, A_t) + \alpha \big[ G_{t:t+n} - Q_{t+n-1}(S_t,A_t) \big],\ 0 \leq t < T]$$

Where $\delta_t =  G_{t:t+n} - Q_{t+n-1}(S_t,A_t) = $ the n-step error.

Backup diagrams for n-step SARSA are shown in the figure below. These backups reverse the order of state and action when compared to the n-step TD prediction algorithm discussed above. 

<img src="img/SARSAN.JPG" alt="Drawing" style="width:400px; height:350px"/>
<center> **Backup Diagram for n-step SARSA** </center>

### Example of N-Step SARSA

The code in the cell below implements the n-step SARSA algorithm. As is the case for the n-step TD algorithm, the bookkeeping is a bit involved. We must account for the cases where fewer than n steps have been executed at the start of an episode or when the terminal state has been reached. You can find further details of the algorithm by reading the code comments.

Execute this code for n=4 and examine the results.

In [17]:
def SARSA_n(policy, episodes, n, alpha = 0.2, gamma = 0.9, epsilon = 0.1, action_index = {'u':0, 'd':1, 'l':2, 'r':3}):
    """
    Function to perform TD(N) policy evaluation.
    """
    ## Initialize the state list and action values
#    action_index = list(range(len(list(policy[0].keys()))))
    states = list(policy.keys())
    n_states = len(states)
    n_actions = len(policy[0].keys())
    q = np.zeros((n_states, n_actions))
    
    for _ in range(episodes):
        ## Initialize variables
        T = float("inf")
        tau = 0
        t = 0
        rewards = []
       
        ## Get the random initial state
        current_state = start_episode(n_states)   
        state = [current_state]
        ## Initial action
        action, s_prime, reward, terminal = take_action(current_state, policy)
        state.append(s_prime)
        
        while(not (tau == T - 1)):
            if(t < T):
                ## Append the reward to the list
                rewards.append(reward)             
                if(terminal): 
                    ## update T if at terminal state
                    T = t + 1
                else: 
                    ## Get the next action state and rewards
                    action_prime, s_prime_prime, reward_prime, terminal_prime = take_action(current_state, policy)
                    state.append(s_prime_prime)
                      
            ## Update tau
            tau = t - n + 1
            
            if(tau > 0):
                G = 0.0
                for i in range(tau + 1, min(tau + n, T)):
                    exponent = i + tau - 1
                    G = G + gamma**exponent * rewards[i]
                if(tau + n < T): G += gamma**n * q[state[tau + n], action_index[action_prime]] 
                q[state[tau], action_index[action]] = q[state[tau], action_index[action]] + alpha * (G - q[state[tau], action_index[action]])    
                
            
            ## Update variables for the next step
            t += 1
            current_state = s_prime
            if(not terminal):
                action = action_prime
                s_prime = s_prime_prime
                reward = reward_prime
                terminal = terminal_prime
    return(q)              
            
Q = SARSA_n(initial_policy, 1000, 4)
print_Q(Q)

          up      down      left     right
0   0.000000  0.000000  0.000000  0.000000
1   0.787506 -0.024938  4.042967  0.317535
2   0.274281  0.514170  3.316137  0.827401
3   0.355605 -0.109827  1.370939 -0.288371
4   1.804363  0.475046  0.489152  0.744228
5   2.235741  1.245969  0.995583  0.696041
6   1.177842  0.279965  1.281592  0.159308
7   1.337316 -0.134156  0.226286 -0.495702
8   1.485867  0.792271  0.574804  0.793659
9   0.675096  0.261018  0.662492  0.353546
10  1.080611  0.458570  0.572490  0.341284
11  0.273256 -0.291059  0.182473 -0.178527
12  0.512627 -0.065658  0.338566  0.266247
13  1.232739  0.317008  1.034362  0.435271
14  1.028716 -0.094236  0.527129  0.041919
15 -0.095867 -0.147639  0.333158 -0.367730


In [18]:
improved_policy = update_policy(initial_policy, Q, 0.1) 
for state in range(16):
    print(improved_policy[state])

{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}


In [14]:
TD_n(improved_policy, 2000, 4).reshape((4,4))

array([[0.        , 2.63969849, 1.78933519, 1.06444613],
       [0.66124227, 0.68596245, 2.01477059, 0.22805499],
       [3.31668986, 1.72310537, 1.51997658, 0.50563337],
       [1.11460711, 1.52031072, 0.61343308, 0.5906463 ]])

### GPI with N-step SARSA

The code in the cell below executes the GPI algorithm using n-step SARSA to evaluate the action values. Details of the algorithm can be found by reading the code comments. 

Execute the code using 50 cycles of 100 episodes and 4-step SARSA and examine the results. 

In [15]:
def SARSA_n_GPI(policy, n, cycles, episodes, goal, alpha = 0.2, gamma = 0.9, epsilon = 0.1):
    ## iterate over GPI cycles
    current_policy = copy.deepcopy(policy)
    for _ in range(cycles):
        ## Evaluate policy with SARSA
        Q = SARSA_n(policy, episodes, n, goal = goal, alpha = alpha, gamma = gamma, epsilon = epsilon)
        
        for s in list(current_policy.keys()): # iterate over all states
            ## Find the index action with the largest Q values 
            ## May be more than one. 
            max_index = np.where(Q[:,s] == max(Q[:,s]))[0]
            
            ## Probabilities of transition
            ## Need to allow for further exploration so don't let any 
            ## transition probability be 0.
            ## Some gymnastics are required to ensure that the probabilities 
            ## over the transistions actual add to exactly 1.0
            neighbors_len = float(Q.shape[0])
            max_len = float(len(max_index))
            diff = round(neighbors_len - max_len,3)
            prob_for_policy = round(1.0/max_len,3)
            adjust = round((epsilon * (diff)), 3)
            prob_for_policy = prob_for_policy - adjust
            if(diff != 0.0):
                remainder = (1.0 - max_len * prob_for_policy)/diff
            else:
                remainder = epsilon
                                                 
            for i, key in enumerate(current_policy[s]): ## Update policy
                if(i in max_index): current_policy[s][key] = prob_for_policy
                else: current_policy[s][key] = remainder   
                    
    return(current_policy)                    
 

SARSA_N_Policy = SARSA_n_GPI(policy, n = 4, cycles = 5, episodes = 1000, goal = 0, alpha = 0.2, epsilon = 0.1)
SARSA_N_Policy

NameError: name 'policy' is not defined

Now, execute the code in the cell below to evaluate the policy you have just computed using the n-step TD algorithm with n = 4. 

In [ ]:
np.array(TD_n(SARSA_N_Policy, episodes = 1000, n = 4, goal = 0, alpha = 0.2, gamma = 0.9)).reshape((4,4))

## N-Step Off-Policy Learning with Importance Sampling

For n-step off-policy learning we update a target policy $\pi(A_t|S_t)$ using samples from a behavior policy $b(A_t|S_t)$. Since the two policies differ, the probabilities of an action given the state will undoubtedly differ. For example, the behavior policy can be exploratory whereas, the target policy is greedy. 

To account for the different probabilities of sampling we reweight by the **importance sampling ratio**. For an n-step algorithm at time step $t$ the importance sampling ratio can be expressed as:

$$\rho_{t:t + n -1} = \prod_{k=\tau}^{min(t + n -1,T-1)} \frac{\pi(A_k|S_k)}{b(A_k|S_k)}$$

The n-step TD update then becomes:

$$V_{t+n}(S_t) = V_{t+n-1}(S_t) + \alpha\ \rho_{t:t+n-1} \big[ G_{t:t+n} - V_{t+n-1}(S_t) \big],\ 0 \leq t < T]$$

And the SARSA update becomes:

$$Q_{t+n}(S_t, A_t) = Q_{t+n-1}(S_t, A_t) + \alpha\ \rho_{t:t+n-1} \big[ G_{t:t+n} - Q_{t+n-1}(S_t,A_t) \big],\ 0 \leq t < T]$$

For both of the above update equations consider the effect of importance sampling ratio. If the action given state is more likely under the target policy that the behavior policy, more weight is given to updating with the error term. However, If the action given state is less likely under the target policy that the behavior policy, less weight is given to updating with the error term. In this way, the weighting by the importance sampling ratio gives the correct updates for the target policy regardless of the transition probabilities of the behavior policy. 

> **NOte:** Considerably more detail on n-step off-policy RL algorithms can be found in Sutton and Barto, second edition, Sections 7.3, 7.4 and 7.5. 

#### Copyright 2018, Stephen F Elston. All rights reserved. 